In [12]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0,1] and adds channel dim
    transforms.Normalize((0.1307,), (0.3081,))  # Mean and std for MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [5]:
def train(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")


In [6]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            preds = output.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [7]:
train(model, train_loader, optimizer, criterion, epochs=5)
evaluate(model, test_loader)


Epoch 1: Loss = 240.8565
Epoch 2: Loss = 107.2108
Epoch 3: Loss = 74.5991
Epoch 4: Loss = 57.0250
Epoch 5: Loss = 44.8866
Test Accuracy: 97.56%


In [15]:
import matplotlib.pyplot as plt

def test_single_image(index=0):
    model.eval()
    image, label = test_dataset[index]
    with torch.no_grad():
        output = model(image.unsqueeze(0).to(device))
        pred = output.argmax(dim=1).item()

    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Predicted: {pred}, True: {label}")
    plt.axis('off')
    plt.show()


In [16]:
from PIL import Image

def predict_custom_image(path):
    image = Image.open(path).convert('L')  # Grayscale
    image = image.resize((28, 28))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        pred = output.argmax(dim=1).item()
    print(f"Predicted Digit: {pred}")
